In [1]:
from coffea import hist
import math

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
import numpy as np
import random
plt.style.use(hep.style.CMS)

import awkward as ak

from matplotlib import colors
POPTS={'norm':colors.LogNorm()}

In [2]:
from data import getData, repackage
ldmx_dict = getData(chunks=True, fnames="/Users/chloeg/Desktop/Work/Fermilab2021/HistData/kshort_pn_4GeV_Jun13_ntuple/*.root")

In [3]:
def extend_array(arr, new_attr, new_attr_name):
    members={n:arr[n] for n in arr.fields}
    members[new_attr_name] = new_attr
    return ak.zip(members)

def add_angle(arr,br=['px','py','pz','e'],name="theta"):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    arr = extend_array(arr, part.theta, name)
    return arr

def get_vector(arr):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    return part

def flat(x,axis=None): # for now must cast while waiting for coffea to catch up
    try:
        return ak.to_numpy(ak.flatten(x,axis=axis)) 
    except:
        return x

In [4]:
def track_smear(kinetic_energy, theta):
    # assume flat uncertainty on the energy (I think that is 6%)
    c_eres = random.gauss(1.,0.06); # flat 10% uncertainty for now
    # assume some uncertainty on the angle (I think that is 2%)
    c_hres = random.gauss(0.,0.02); # 1 degrees
    return (c_eres*kinetic_energy, abs(c_hres+theta))

def piecewise_smear(pz):
    c_eres_array = []
    for part in pz:
        for element in part:
            if element <= 50:
                c_eres = random.gauss(1,0.0404);
            elif element <= 100: 
                c_eres = random.gauss(1,0.0410);
            elif element <= 250: 
                c_eres = random.gauss(1,0.0415);
            elif element <= 500: 
                c_eres = random.gauss(1,0.0420);
            elif element <= 1000: 
                c_eres = random.gauss(1,0.0435);
            elif element <= 2000:  
                c_eres = random.gauss(1,0.0477);
            elif element <= 3000:  
                c_eres = random.gauss(1,0.0523);
            elif element <= 4000:  
                c_eres = random.gauss(1,0.0573);            
            else:    
                c_eres = random.gauss(1,0.06);
            c_eres_array=np.append(c_eres_array, [c_eres])
    return (c_eres_array)

In [5]:
hists = {}
hists["sim_pion_e1"] = hist.Hist("Sim Kaons",
                                hist.Bin("e", r"$\pi^\pm$ Energy [MeV]", 80, 0, 1200), 
                            )  
hists["sim_pion_e2"] = hist.Hist("Sim Kaons",
                                hist.Bin("e", r"$\pi^\pm$ Energy [MeV]", 80, 0, 1200), 
                            )  
hists["sim_pion_e3"] = hist.Hist("Sim Kaons",
                                hist.Bin("e", r"$\pi^\pm$ Energy [MeV]", 80, 0, 1200), 
                            )
hists["sim_pion_e4"] = hist.Hist("Sim Kaons",
                                hist.Bin("e", r"$\pi^\pm$ Energy [MeV]", 80, 0, 1200), 
                            )
hists["sim_pion_e5"] = hist.Hist("Sim Kaons",
                                hist.Bin("e", r"$\pi^\pm$ Energy [MeV]", 80, 0, 1200), 
                            )

hists["sim_pion_mass1"] = hist.Hist("Sim Kaons",
                                hist.Bin("e", r"Reconstructed $K_s$ Mass [MeV]", 80, 400, 600), 
                            )  
hists["sim_pion_mass2"] = hist.Hist("Sim Kaons",
                                hist.Bin("e", r"Reconstructed $K_s$ Mass [MeV]", 80, 400, 600), 
                            )  
hists["sim_pion_mass3"] = hist.Hist("Sim Kaons",
                                hist.Bin("e", r"Reconstructed $K_s$ Mass [MeV]", 80, 400, 600), 
                            ) 
hists["sim_pion_mass4"] = hist.Hist("Sim Kaons",
                                hist.Bin("e", r"Reconstructed $K_s$ Mass [MeV]", 80, 400, 600), 
                            ) 
hists["sim_pion_mass5"] = hist.Hist("Sim Kaons",
                                hist.Bin("e", r"Reconstructed $K_s$ Mass [MeV]", 80, 400, 600), 
                            ) 
hists["sim_kaone_hist"] = hist.Hist("Sim Pions",     
                                hist.Bin("k", r"Reconstructed $K_s$ Kinetic Energy [MeV]", 40, 0, 3500),
                                hist.Bin("e", r"Electron Energy [MeV]", 40, 0, 1500),
                            )


In [6]:
def ProcessChunk(chunk, hists):
    ldmx_events = repackage(chunk)
    
    #constant definitions
    kshort_pdgid = 310
    chargedpion_pdgid = 211
    pion_mass = 139
    kaon_mass = 497.648
 
    #select events
    sim_kaon = ldmx_events['Sim_Kaon']
    sim_kaon_dau1 = ldmx_events['Sim_Kaon_dau1']
    sim_kaon_dau2 = ldmx_events['Sim_Kaon_dau2']
    
    #sim kaon masks
    k_mask = (abs(sim_kaon.pdgID)==kshort_pdgid)
    kshort_mask = (abs(sim_kaon_dau2.mompdgID)==kshort_pdgid)
        #same as kshort_pi_mask
    pidau_mask = (abs(sim_kaon_dau2.pdgID)==chargedpion_pdgid)

    #define vectors
    pi1 = get_vector(sim_kaon_dau1[pidau_mask & kshort_mask])
    pi2 = get_vector(sim_kaon_dau2[pidau_mask & kshort_mask])
    pisum = (pi1 + pi2)
    
    #define vector kinematics
    new_pz1 = piecewise_smear(pi1.pz)*flat(pi1.pz)
    new_psq1 = flat(pi1.px)**2+flat(pi1.py)**2+new_pz1**2
    new_pz2 = piecewise_smear(pi2.pz)*flat(pi2.pz)
    new_psq2 = flat(pi2.px)**2+flat(pi2.py)**2+new_pz2**2
    
    #theta masks
    theta_mask1 = ((40*math.pi/180) >= (flat(pi2.theta)))
    theta_mask2 = ((40*math.pi/180) >= (pi2.theta))
    
    #create smeared events
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)
    pi1_track_smeared =  ak.zip({"x": flat(pi1.px),
                    "y": flat(pi1.py),
                    "z": new_pz1,
                    "t": (np.sqrt(new_psq1+flat(pi1.mass)**2)),
                    },
                    with_name="LorentzVector")
    pi2_track_smeared =  ak.zip({"x": flat(pi2.px),
                    "y": flat(pi2.py),
                    "z": new_pz2,
                    "t": (np.sqrt(new_psq2+flat(pi2.mass)**2)),
                    },
                    with_name="LorentzVector")
    pisum_track_smeared =(pi1_track_smeared+pi2_track_smeared)
    pisum_smeared_e_append = np.append(flat(pi1_track_smeared[theta_mask1].t), flat(pi2_track_smeared[theta_mask1].t))
    
    #percent masks
    mask_15 = (-4.0*flat(sim_kaon_dau2[theta_mask2 & pidau_mask & kshort_mask].mome-kaon_mass)+2800) <= flat(sim_kaon_dau2[theta_mask2 & pidau_mask & kshort_mask].electrone)
    mask_30 = (-4.0*flat(sim_kaon_dau2[theta_mask2 & pidau_mask & kshort_mask].mome-kaon_mass)+2050) <= flat(sim_kaon_dau2[theta_mask2 & pidau_mask & kshort_mask].electrone)
    mask_60 = (-4.0*flat(sim_kaon_dau2[theta_mask2 & pidau_mask & kshort_mask].mome-kaon_mass)+1300) <= flat(sim_kaon_dau2[theta_mask2 & pidau_mask & kshort_mask].electrone)
    mask_80 = (-4.0*flat(sim_kaon_dau2[theta_mask2 & pidau_mask & kshort_mask].mome-kaon_mass)+850) <= flat(sim_kaon_dau2[theta_mask2 & pidau_mask & kshort_mask].electrone)
    
    #acceptance_d = flat(flat(pisum_track_smeared[testmask].mass).shape)
    #acceptance_n = flat(flat(pisum_track_smeared.mass).shape)
    #acceptance = flat(acceptance_d / acceptance_n)
    
    #fill histograms
    hists["sim_kaone_hist"].fill(
                                 k = flat(pisum_track_smeared[theta_mask1].t) - kaon_mass,
                                 e = flat(sim_kaon_dau2[kshort_mask][theta_mask2].electrone),
                                )
    
    hists["sim_pion_mass1"].fill(
                                 e=flat(pisum_track_smeared.mass),
                                )
    hists["sim_pion_mass2"].fill(
                                 e=flat(pisum_track_smeared[mask_15].mass),
                                )
    hists["sim_pion_mass3"].fill(
                                 e=flat(pisum_track_smeared[mask_30].mass),
                                )
    hists["sim_pion_mass4"].fill(
                                 e=flat(pisum_track_smeared[mask_60].mass),
                                )
    hists["sim_pion_mass5"].fill(
                                 e=flat(pisum_track_smeared[mask_80].mass),
                                )
    hists["sim_pion_e1"].fill(
                                e=flat(pisum_smeared_e_append),
                                ) 
    pisum_smeared_e_append_15 = np.append(flat(pi1_track_smeared[theta_mask1][mask_15].t), flat(pi2_track_smeared[theta_mask1][mask_15].t))
    hists["sim_pion_e2"].fill(
                                e=flat(pisum_smeared_e_append_15),
                                ) 
    pisum_smeared_e_append_30 = np.append(flat(pi1_track_smeared[theta_mask1][mask_30].t), flat(pi2_track_smeared[theta_mask1][mask_30].t))
    hists["sim_pion_e3"].fill(
                                e=flat(pisum_smeared_e_append_30),
                                ) 
    pisum_smeared_e_append_60 = np.append(flat(pi1_track_smeared[theta_mask1][mask_60].t), flat(pi2_track_smeared[theta_mask1][mask_60].t))
    hists["sim_pion_e4"].fill(
                                e=flat(pisum_smeared_e_append_60),
                                ) 
    pisum_smeared_e_append_80 = np.append(flat(pi1_track_smeared[theta_mask1][mask_80].t), flat(pi2_track_smeared[theta_mask1][mask_80].t))
    hists["sim_pion_e5"].fill(
                                e=flat(pisum_smeared_e_append_80),
                                ) 
    print((flat(flat(pisum_track_smeared.t) - kaon_mass).shape))
    return #acceptance

In [7]:
nchunk = 0
percent = 0
for chunk in ldmx_dict:
    nchunk += 1
    #returna = ProcessChunk(chunk, hists)
    #percent += 100*returna[0]
    ProcessChunk(chunk, hists)
    print('process', nchunk) 
    #print('percent', percent/nchunk)

(236,)
process 1
(206,)
process 2
(220,)
process 3
(244,)
process 4
(222,)
process 5
(219,)
process 6
(261,)
process 7
(236,)
process 8
(268,)
process 9
(231,)
process 10
(231,)
process 11


KeyboardInterrupt: 

In [ ]:
print(hists["sim_pion_mass1"])
fig, ax = plt.subplots(1,2,figsize=(24,9), constrained_layout=True)

 #this is 100% of the data with theta cut
hist.plot1d(hists["sim_pion_mass1"],ax=ax[0],clear=False);
hist.plot1d(hists["sim_pion_mass5"],ax=ax[0],clear=False);
hist.plot1d(hists["sim_pion_mass4"],ax=ax[0],clear=False);
hist.plot1d(hists["sim_pion_mass3"],ax=ax[0],clear=False);
hist.plot1d(hists["sim_pion_mass2"],ax=ax[0],clear=False);
leg = ax[0].legend([r'100%', r'80%', r'60%', r'30%', r'15%' ])

hist.plot1d(hists["sim_pion_e1"],ax=ax[1],clear=False);
hist.plot1d(hists["sim_pion_e5"],ax=ax[1],clear=False);
hist.plot1d(hists["sim_pion_e4"],ax=ax[1],clear=False);
hist.plot1d(hists["sim_pion_e3"],ax=ax[1],clear=False);
hist.plot1d(hists["sim_pion_e2"],ax=ax[1],clear=False);
leg = ax[1].legend([r'100%', r'80%', r'60%', r'30%', r'15%' ])


print(hists["sim_kaone_hist"])
fig, ax = plt.subplots()
hist.plot2d(hists["sim_kaone_hist"], xaxis='k', ax=ax, patch_opts=POPTS);
ax.text(2900, 1400,r'$\theta$<40$^\circ$')


x = np.arange(0, 3250, 1)
line15 = -4.0* x + 2800
line30 = -4.0* x + 2050
line60 = -4.0* x + 1300
line80 = -4.0* x + 850
ax.plot(x, line15, '-')
ax.plot(x, line30, '-')
ax.plot(x, line60, '-')
ax.plot(x, line80, '-')

In [ ]:
print(hists["sim_kaone_hist"])
fig, ax = plt.subplots()
hist.plot2d(hists["sim_kaone_hist"], xaxis='k', ax=ax, patch_opts=POPTS);

